In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
        #print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv("/kaggle/input/plant-pathology-2021-fgvc8/train.csv")
Submission = pd.read_csv("/kaggle/input/plant-pathology-2021-fgvc8/sample_submission.csv")
Prefix="/kaggle/input/plant-pathology-2021-fgvc8/train_images/"

print(train)

In [ ]:
#BORRAR
train=train.loc[0:1500]
print(train.shape)

In [ ]:
print(train["labels"].value_counts())

In [ ]:
#Convert categorical variable into dummy/indicator variables
y_train=pd.get_dummies(train["labels"])
print(y_train.head())

In [ ]:
y_train["scab"] = np.where(y_train['scab frog_eye_leaf_spot']==1,1,y_train["scab"])
y_train["frog_eye_leaf_spot"] = np.where(y_train['scab frog_eye_leaf_spot']==1,1,y_train["frog_eye_leaf_spot"])

y_train["scab"] = np.where(y_train['scab frog_eye_leaf_spot complex']==1,1,y_train["scab"])
y_train["frog_eye_leaf_spot"] = np.where(y_train['scab frog_eye_leaf_spot complex']==1,1,y_train["frog_eye_leaf_spot"])
y_train["complex"] = np.where(y_train['scab frog_eye_leaf_spot complex']==1,1,y_train["complex"])

y_train["frog_eye_leaf_spot"] = np.where(y_train['frog_eye_leaf_spot complex']==1,1,y_train["frog_eye_leaf_spot"])
y_train["complex"] = np.where(y_train['frog_eye_leaf_spot complex']==1,1,y_train["complex"])

y_train["frog_eye_leaf_spot"] = np.where(y_train['rust frog_eye_leaf_spot']==1,1,y_train["frog_eye_leaf_spot"])
y_train["rust"] = np.where(y_train['rust frog_eye_leaf_spot']==1,1,y_train["rust"])

y_train["complex"] = np.where(y_train['rust complex']==1,1,y_train["complex"])
y_train["rust"] = np.where(y_train['rust complex']==1,1,y_train["rust"])

y_train["complex"] = np.where(y_train['powdery_mildew complex']==1,1,y_train["complex"])
y_train["powdery_mildew"] = np.where(y_train['powdery_mildew complex']==1,1,y_train["powdery_mildew"])

In [ ]:
y_train=y_train.loc[:,['complex', 'frog_eye_leaf_spot', 'healthy', 'powdery_mildew', 'rust',"scab"]]
print(y_train.head())

In [ ]:
# Put a number between 0 and 18632 to visualizate

Numb=274

location=Prefix + train["image"][Numb]
print(location)

%pylab inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

img = mpimg.imread(location)
imgplot = plt.imshow(img)
plt.show()
print(train["labels"][Numb])

In [ ]:
n_train=train.shape[0]
print(n_train)

In [ ]:
# Defining the bath size and putting in variables the rest of the data.
IMAGE_WIDTH = 512
IMAGE_HEIGHT = 512
IMAGE_CHANNELS = 3
IMAGE_SIZE = (IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNELS)

BATCH_SIZE = n_train#/258

In [ ]:
from tqdm import tqdm

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from tqdm import tqdm

In [ ]:
train_image = []
for i in tqdm(range(0,n_train)):#train.shape[0])):
    location=Prefix + train["image"][i]
    img = image.load_img(location, target_size=(IMAGE_HEIGHT, IMAGE_WIDTH,IMAGE_CHANNELS), grayscale=False)
    img = image.img_to_array(img)
    img = img/255
    train_image.append(img)
X = np.array(train_image)

In [ ]:
import math
import numpy as np
import h5py
import matplotlib.pyplot as plt
import scipy
from PIL import Image
from scipy import ndimage

import tensorflow as tf
from tensorflow.python.framework import ops

from sklearn.model_selection import train_test_split

%matplotlib inline
np.random.seed(1)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Dropout, Flatten, Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
early_stopping = EarlyStopping(
    monitor = "val_loss", 
    min_delta = 0.0008, 
    patience = 40, 
    restore_best_weights = True, 
    verbose = 1
)

callbacks = [early_stopping]

In [ ]:
model = Sequential([
  Conv2D(filters = 32, kernel_size = (5,5), padding = "same", input_shape = IMAGE_SIZE),
  Activation("relu"),
  MaxPooling2D(),
  Dropout(0.45),
  Conv2D(filters = 24, kernel_size = (5, 5), padding = "same"),
  Activation("relu"),
  MaxPooling2D(),
  Dropout(0.45),
  Conv2D(filters = 128, kernel_size = (5, 5), padding = "same"),
  Activation("relu"),
  MaxPooling2D(),
  Conv2D(filters = 256, kernel_size = (5, 5), padding = "same"),
  Activation("relu"),
  MaxPooling2D(),
  Dropout(0.45),
  Flatten(),
  Dense(32),
  Activation("relu"),
  Dropout(0.45),
  Dense(6, activation = "softmax"),
])

model.summary()

In [ ]:
model.compile(optimizer = "adam", loss = "categorical_crossentropy", metrics = ["accuracy"])

In [ ]:
y = to_categorical(y_train)
y=y[:,:,1]
print(y)

In [ ]:

X_train, X_test, y_train2, y_test = train_test_split(X, y, random_state=42, test_size=0.85)

print(y_train2.shape)
print(X_train.shape)

In [ ]:
Model1=model.fit(
    x = X_train,
    y= y_train2,
    steps_per_epoch = n_train // BATCH_SIZE,
    epochs = 250, 
    validation_data = (X_test, y_test), 
    callbacks = callbacks
)

In [ ]:
hist = Model1.history
fig, ax = plt.subplots(figsize = (15,8))   
ax.grid(True)
ax = fig.add_subplot(1, 2, 1)
ax.plot(hist['loss'], lw=2)
ax.set_title('Training loss', size=15)
ax.set_xlabel('Epoch', size=15)
ax.tick_params(axis='both', which='major', labelsize=15)

ax = fig.add_subplot(1, 2, 2)
ax.plot(hist['accuracy'], lw=2)
ax.set_title('Training accuracy', size=15)
ax.set_xlabel('Epoch', size=15)
ax.tick_params(axis='both', which='major', labelsize=15)
plt.tight_layout()
plt.show()

In [ ]:
predictions = model.predict(X_test)
print(predictions)

In [ ]:
predictions = np.argmax(predictions, axis = 1)
print(predictions)

In [ ]:
predictions = model.predict(X_train)
print(predictions)